In [3]:
import pandas as pd
df = pd.read_csv('wenben.csv')
X = df[['xinxi']]
y = df.leixing

In [4]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
X['cutted_xinxi'] = X.xinxi.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\mazheng\AppData\Local\Temp\jieba.cache
Loading model cost 0.892 seconds.
Prefix dict has been built succesfully.


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [7]:
stop_words_file = "stopwords.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer



vect = CountVectorizer(token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',stop_words=frozenset(stopwords))

term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_xinxi).toarray(), columns=vect.get_feature_names())

term_matrix.head()

,bk6997,b座,caoliubb,cn,cp,e19899,gofun,http,ktv,led,...,鼓风机,齐抓共管,龙居六区,龙山,龙岩,龙泉,龙湾,龙潭,龙虾,龙门
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

from sklearn.pipeline import make_pipeline
pipe = make_pipeline(vect, nb)

from sklearn.cross_validation import cross_val_score
cross_val_score(pipe, X_train.cutted_xinxi, y_train, cv=5, scoring='accuracy').mean()

0.88130249332586907

In [16]:
pipe.fit(X_train.cutted_xinxi, y_train)
pipe.predict(X_test.cutted_xinxi)

y_pred = pipe.predict(X_test.cutted_xinxi)

from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

metrics.confusion_matrix(y_test, y_pred)

array([[314,  10],
       [ 40,   2]], dtype=int64)

In [21]:
import numpy as np
empty = pd.DataFrame(columns=["xinxi"])
result = empty.append(pd.DataFrame({'xinxi': [" 天津市津南区百万路延长线百利恒大酒店，五楼六楼房间内有小姐提供色情服务，上到五楼之后会有服务员主动来询问"]}), ignore_index=True)

result['cutted_xinxi'] = result.xinxi.apply(chinese_word_cut)
jieguo = pipe.predict(result.cutted_xinxi)
yucezhi = np.array(jieguo)
yuce = yucezhi[0]
yuce


1